# Thermodynamic Derivatives

In [6]:
import teqp
import numpy as np

In [7]:
Tc_K = [300]
pc_Pa = [4e6]
acentric = [0.01]
model = teqp.canonical_PR(Tc_K, pc_Pa, acentric)

In [8]:
z = np.array([1.0])
model.get_Ar01(300,300,z)

-0.06836660379313926